In [1]:
import sqlite3
conn = sqlite3.connect('lab4.db')

c = conn.cursor()

In [2]:
# c.execute('''DROP TABLE gene;''')
# c.execute('''DROP TABLE enzyme;''')
# c.execute('''DROP TABLE en_pa;''')
# c.execute('''DROP TABLE pathway;''')

In [3]:
c.execute('''CREATE TABLE gene
             (id INT, name TEXT, sequence TEXT, eid INT )''')

c.execute('''CREATE TABLE enzyme
            (id INT, name TEXT, function TEXT, ecnum TEXT )''')

c.execute('''CREATE TABLE en_pa (eid INT, pid INT, priority INT )''')

c.execute('''CREATE TABLE pathway
             (id INT, name TEXT, description TEXT )''')

In [4]:
# Add pathway

pa_file = open("lab4_pathway_data")

name = ''
description = ''

iterator = 1

for i, line in enumerate(pa_file):

    line = line.replace('\n', '')
    
    if '#' in line:
        continue
        
    if i%3 is 0:
        name = line
    elif i%3 is 1:
        description = line
    elif i%3 is 2:
        c.execute("INSERT INTO pathway (id, name, description) VALUES ("+ str(iterator) + ", '" + name + "', '" + description + "' )" )        
        iterator+=1

conn.commit()


In [5]:
# Add enzyme

enz_file = open("lab4_enzyme_data")

name = ''
pathorder = []
function = ''
ECnum = ''

iterator = 1


for i, line in enumerate(enz_file):

    line = line.replace('\n', '')
    
    if '#' in line:
        continue
    if i%5 is 0:
        name = line
    elif i%5 is 1:
        pathorder = line.split(',')
        for i, item in enumerate(pathorder):
            pathorder[i] = pathorder[i].split('|')
            pathorder[i][1] = int(pathorder[i][1])
    elif i%5 is 2:
        function = line
    elif i%5 is 3:
        ECnum = line
    elif i%5 is 4:
        c.execute("INSERT INTO enzyme (id, name, function, ecnum) VALUES ("+ str(iterator) + ", '" + name + "', '" + function +"', '"+ ECnum + "' )" )
        
        ### TODO ###
        for rep in range(len(pathorder)):
            c.execute("INSERT INTO en_pa (eid, pid, priority) VALUES ("+ str(iterator) + "," + "(select id from pathway where name = "+"'"+pathorder[rep][0]+"')"+" ,"+str(pathorder[rep][1])+")")
            
        iterator+=1
        
conn.commit()

In [6]:
# Add Gene



conn.commit()

In [7]:
# print table

import pandas as pd

#print (pd.read_sql_query("SELECT * FROM gene;",conn))
print (pd.read_sql_query("SELECT * FROM enzyme;", conn))
print (pd.read_sql_query("SELECT * FROM pathway;", conn))
print (pd.read_sql_query("SELECT * FROM en_pa;", conn))

    id                               name  \
0    1          Triosephosphate isomerase   
1    2            Phosphoglycerate kinase   
2    3    Phosphoenolpyruvate carboxylase   
3    4                    Pyruvate kinase   
4    5  Glucose-6-phosphate dehydrogenase   
5    6                   Gluconolactonase   
6    7   6-phosphogluconate dehydrogenase   
7    8                      Transaldolase   
8    9            Succinate dehydrogenase   
9   10                           Fumarase   
10  11               Malate dehydrogenase   
11  12                   Citrate synthase   

                                             function        ecnum  
0   catalyzes the reversible interconversion of th...   EC 5.3.1.1  
1   catalyzes the reversible transfer of a phospha...   EC 2.7.2.3  
2   an enzyme in the family of carboxy-lyases foun...  EC 4.1.1.31  
3   It catalyzes the transfer of a phosphate group...  EC 2.7.1.40  
4   D-glucose 6-phosphate + NADP⁺ ⇌ 6-phospho-D-gl...  EC 1.1.1.49  


In [31]:
# view by enzyme priority in each pathway SORTED BY ORDER!!

print(pd.read_sql_query("SELECT enzyme.id, enzyme.name FROM enzyme INNER JOIN en_pa on enzyme.id = en_pa.eid WHERE en_pa.pid = 2 and en_pa.priority>0 ORDER BY en_pa.priority ASC ;", conn))

# sort enzymes by iwhere pid=1 and have priority information(priority>0)


   id                               name
0   5  Glucose-6-phosphate dehydrogenase
1   6                   Gluconolactonase
2   7   6-phosphogluconate dehydrogenase
3   8                      Transaldolase
